In [1]:
import numpy as np
from tensorflow import keras
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
LATENT_DIM=10
BATCH_SIZE=256
HALF_BATCH=BATCH_SIZE//2
EPOCHS=125

In [3]:
(x1, _), (x2, _) = keras.datasets.mnist.load_data()
X=np.vstack((x1,x2))
X=X/255.0
X=np.expand_dims(X,axis=-1)
X.shape

(70000, 28, 28, 1)

In [4]:
def save_images(name, n=10):
    images=g_model.predict(np.random.randn(n**2,100).astype('float32'))
    for i in range(n**2):
        temp=images[i].reshape((28,28))
        temp=temp/255.0
        plt.subplot(n, n, 1 + i)
        plt.axis('off')
        plt.imshow(temp, cmap='gray_r')
    plt.savefig(f'images\\{name}.png')
    plt.close()

In [5]:
def make_discriminator():
	model = keras.models.Sequential()
	model.add(keras.layers.Conv2D(64, (3,3), strides=(2, 2), padding='same', input_shape=(28,28,1)))
	model.add(keras.layers.LeakyReLU(alpha=0.25))
	model.add(keras.layers.Dropout(0.4))
	model.add(keras.layers.Conv2D(64, (3,3), strides=(2, 2), padding='same'))
	model.add(keras.layers.LeakyReLU(alpha=0.25))
	model.add(keras.layers.Dropout(0.4))
	model.add(keras.layers.Flatten())
	model.add(keras.layers.Dense(128))
	model.add(keras.layers.LeakyReLU(alpha=0.25))
	model.add(keras.layers.Dropout(0.25))
	model.add(keras.layers.Dense(1, activation='sigmoid'))
	opt = keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model
 
def make_generator(latent_dim=LATENT_DIM):
	model = keras.models.Sequential()
	n_nodes = 128 * 7 * 7
	model.add(keras.layers.Dense(n_nodes, input_dim=latent_dim))
	model.add(keras.layers.LeakyReLU(alpha=0.2))
	model.add(keras.layers.Reshape((7, 7, 128)))
	model.add(keras.layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(keras.layers.LeakyReLU(alpha=0.2))
	model.add(keras.layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(keras.layers.LeakyReLU(alpha=0.2))
	model.add(keras.layers.Conv2D(1, (7,7), activation='sigmoid', padding='same'))
	return model

def make_gan(gen, dis):
	d_model.trainable = False
	model = keras.models.Sequential()
	model.add(gen)
	model.add(dis)
	opt = keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

d_model = make_discriminator()
g_model = make_generator()
gan_model = make_gan(g_model, d_model)



In [7]:
save_images(0)
for epoch in range(EPOCHS):
    print(f'Epoch:{epoch+1} ',end='')
    for batch in tqdm(range(X.shape[0]//BATCH_SIZE)):

        X_real = X[np.random.randint(0,X.shape[0],HALF_BATCH)]
        y_real = np.ones((HALF_BATCH,1))
        
        X_fake= g_model.predict(np.random.randn(HALF_BATCH,LATENT_DIM))
        y_fake = np.zeros((HALF_BATCH,1))
        
        X_d = np.vstack((X_real, X_fake))
        y_d = np.vstack((y_real, y_fake))
        
        d_loss, _ = d_model.train_on_batch(X_d, y_d)
        for _ in range(2):
            X_gan = np.random.randn(BATCH_SIZE,LATENT_DIM)
            y_gan = np.ones((BATCH_SIZE, 1))
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
    if (epoch+1)%5==0:
        g_model.save('models\\generator.h5')
        d_model.save('models\\discriminator.h5')
    save_images(epoch+1)
    print(f'Discriminator Loss:{d_loss}, Generator Loss:{g_loss}')

Epoch:26 

100%|██████████| 273/273 [01:23<00:00,  3.27it/s]


Discriminator Loss:0.6800878047943115, Generator Loss:0.7284817695617676
Epoch:27 

100%|██████████| 273/273 [01:34<00:00,  2.90it/s]


Discriminator Loss:0.6886552572250366, Generator Loss:0.6845147013664246
Epoch:28 

...

Discriminator Loss:0.6828200817108154, Generator Loss:0.7067338228225708
Epoch:124 

100%|██████████| 273/273 [01:20<00:00,  3.40it/s]


Discriminator Loss:0.685991108417511, Generator Loss:0.7131604552268982
Epoch:125 

100%|██████████| 273/273 [01:18<00:00,  3.47it/s]

Discriminator Loss:0.6856302618980408, Generator Loss:0.7151373624801636


In [8]:
g_model.save('models\\generator.h5')
d_model.save('models\\discriminator.h5')